In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import f1_score

sns.set_theme(context = 'notebook', style = 'darkgrid')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Métodos de Classificação II

Vamos avançar nossos estudos sobre métodos de classificação analisando dois novos algoritmos (o *Perceptron de Múltiplas Camadas* e os *Modelos de Ensemble*). Além disso teremos o primeiro contato com os conceitos de *underfitting* e *overfitting*, e como esses conceitos se relacionam com a **complexidade** dos modelos de ML.

In [ ]:
tb_hotel_train = pd.read_csv("data/tb_hotel_train_clean.csv")
tb_hotel_test = pd.read_csv("data/tb_hotel_test_clean.csv")
tb_hotel_train.head()


In [ ]:
x_var = ["lead_time", "adr"]
X_train = tb_hotel_train[x_var]
X_test = tb_hotel_test[x_var]
y_train = tb_hotel_train["is_cancelled"]
y_test = tb_hotel_test["is_cancelled"]


Para facilitar a visualização dos nossos modelos continuaremos nos restringindo à duas variáveis: `lead_time` e `adr`. Vamos criar dois DataFrames para visualizar separadamente os efeitos de cada variável sobre **a função de probabilidade** estimada por cada algoritmo:

In [ ]:
lead_time_simul = list(np.linspace(-1, 3, 100)) * 3
adr_simul = [-1] * 100 + [0] * 100 + [1] * 100
tb_simul_lt = pd.DataFrame({'lead_time' : lead_time_simul, 'adr' : adr_simul})
tb_simul_lt.head()

In [ ]:
adr_simul = list(np.linspace(-2, 2, 100)) * 3
lead_time_simul = [-1] * 100 + [0] * 100 + [1] * 100
tb_simul_adr = pd.DataFrame({'lead_time' : lead_time_simul, 'adr' : adr_simul})
tb_simul_adr.head()

Vamos criar um DataFrame com os dados originais de teste para guardarmos as diferentes previsões de nossos modelos:

In [ ]:
tb_fits = X_test.copy()
tb_fits["is_cancelled"] = y_test

# Redes Neurais

Redes neurais são uma classe de modelo preditivo que surgiu a partir da pesquisa sobre o funcionamento das células neuronais do cérebro - desenvolvida inicialmente por McCulloh e Pitts na década de 40. O modelo mais comum de rede neural, o **perceptron de múltiplas camadas (MLP)** foi desenvolvido em 1958 pelo psicológo Rosenblatt.

Os **MLPs** são caracterizados por dois atributos fundamentais:

* Número de neurônios por camada;
* Função de Ativação.

![MLP](images/multipercep.jpg)

Vamos criar um MLP para resolver nosso problema de classificação, variando o tamanho de nossa rede neural e analisando o resultado de nossas previsões:

In [ ]:
nn_fit = MLPClassifier(hidden_layer_sizes=(4, 4), activation = 'relu', max_iter=1000)
nn_fit.fit(X_train, y_train)

In [ ]:
tb_fits["pred_prob_nn"] = nn_fit.predict_proba(X_test)[:, 1]

In [ ]:

fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.scatterplot(
    data=tb_fits, 
    x="lead_time", y="pred_prob_nn", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.scatterplot(
    data=tb_fits, 
    x="adr", y="pred_prob_nn", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');



Vamos visualizar a não-linearidade das redes neurais através das superficies de decisão:

In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_fits[["lead_time", "adr"]]),
    np.array(tb_fits["is_cancelled"]),
    nn_fit,
    scatter_kwargs={"alpha": 0.001},
)


In [ ]:
tb_simul_lt['pred_prob_nn1'] = nn_fit.predict_proba(tb_simul_lt[x_var])[:,1]
tb_simul_adr['pred_prob_nn1'] = nn_fit.predict_proba(tb_simul_adr[x_var])[:,1]
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.lineplot(
    data=tb_simul_lt, 
    x="lead_time", y="pred_prob_nn1", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.lineplot(
    data=tb_simul_adr, 
    x="adr", y="pred_prob_nn1", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');

Vamos criar uma nova rede neural como a mesma função de ativação (`relu`) mas aumentando o número de neurônios por camada e o número de camadas:

In [ ]:
nn_fit_2 = MLPClassifier(hidden_layer_sizes=(20,20,20), activation = 'relu')
nn_fit_2.fit(X_train, y_train)

tb_fits["pred_prob_nn2"] = nn_fit_2.predict_proba(X_test)[:, 1]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.scatterplot(
    data=tb_fits, 
    x="lead_time", y="pred_prob_nn2", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.scatterplot(
    data=tb_fits, 
    x="adr", y="pred_prob_nn2", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');

Já podemos ver que os padrões encontrados pela nossa rede neural se tornaram mais complexos: podemos visualizar isso de forma mais direta através da superficie de decisão:

In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_fits[["lead_time", "adr"]]),
    np.array(tb_fits["is_cancelled"]),
    nn_fit_2,
    scatter_kwargs={"alpha": 0.001},
);


Quanto mais irregulares a separação entre as duas classes, mais complexo é o modelo!

In [ ]:
tb_simul_lt['pred_prob_nn2'] = nn_fit_2.predict_proba(tb_simul_lt[x_var])[:,1]
tb_simul_adr['pred_prob_nn2'] = nn_fit_2.predict_proba(tb_simul_adr[x_var])[:,1]
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.lineplot(
    data=tb_simul_lt, 
    x="lead_time", y="pred_prob_nn2", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.lineplot(
    data=tb_simul_adr, 
    x="adr", y="pred_prob_nn2", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');

Como podemos ver, os efeitos das duas variáveis de nosso modelo são mais irregulares na segunda rede neural. A capacidade de representar superficies de decisão complexas é uma das grandes vantagens que os algoritmos de ML tem sobre métodos mais tradicionais como a regressão logística. No entanto, a complexidade introduz um novo problema: overfitting.

# Complexidade e Overfitting

Para avaliarmos melhor a relação entre complexidade e overfitting, vamos carregar mais variáveis de nosso dataset:

In [ ]:
train_of = pd.read_csv('data/tb_hotel_train_overfit.csv')
test_of = pd.read_csv('data/tb_hotel_test_overfit.csv')

X_train_of = train_of.drop('is_cancelled', axis = 1)
X_test_of = test_of.drop('is_cancelled', axis = 1)
y_train_of = train_of['is_cancelled']
y_test_of = test_of['is_cancelled']

Como vimos no começo do Módulo III, podemos dividir a utilização dos algoritmos de ML em duas etapas:

* **Aprendizagem**, onde o algoritmo *aprende* as relações entre nossas *features* e a *variável resposta* utilizando dados históricos (representados pelo conjunto de treinamento);
* **Predição**, onde utilizamos os padrões *aprendidos* pelo algoritmo para realizar projeções sobre novos dados a partir de nossos *features* (represetado pelo conjunto de teste).

A **fase de aprendizagem** consiste na otimização do erro de projeção sobre o conjunto de treinamento - o modelo ajusta gradualmente seus coeficientes buscando melhorar a cada etapa seu erro de projeção sobre os dados históricos. Conforme aumentamos a **complexidade** do modelo essa otimização torna-se cada vez mais eficiente. 

Isso não significa, necessariamente, que o erro de previsão do modelo melhorará! Conforme a **complexidade** aumenta, o modelo perde a **capacidade de generalização**: ao invés de *encontrar padrões* nos dados históricos ele aprende regras que se aplicam somente às observações do conjunto de treinamento.

Vamos utilizar uma árvore de decisão para visualizar este processo:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

As árvores de decisão tem um parâmetro de complexidade muito simples, a **profundida máxima**. Vamos utilizar um `loop for` para construir árvores de decisão de diferentes profundidades e avaliar seu erro sobre o conjunto de treinamento e teste.

In [ ]:
max_depth = [int(x) for x in np.linspace(2, 40, 20)]

d_list = []
f1_train_list = []
f1_test_list = []

for d in max_depth:
    rf_fit = DecisionTreeClassifier(max_depth= d)
    rf_fit.fit(X_train_of, y_train_of)
    y_pred_test = rf_fit.predict(X_test_of)
    y_pred_train = rf_fit.predict(X_train_of)

    f1_test = np.round(f1_score(y_test_of, y_pred_test), 4)
    f1_train = np.round(f1_score(y_train_of, y_pred_train), 4)


    d_list.append(d)
    f1_train_list.append(f1_train)
    f1_test_list.append(f1_test)

In [ ]:
tb_rf_fit = pd.DataFrame(
    {
        'depth' : d_list,
        'f1_train' : f1_train_list,
        'f1_test' : f1_test_list
    }
)
tb_rf_fit['diff_error'] = tb_rf_fit['f1_train'] - tb_rf_fit['f1_test']
tb_rf_fit.head()

Agora vamos comparar a evolução do erro sobre os dois conjuntos, teste e treinamento, para visualizar o impacto da complexidade (representada pela profundidade) sobre underfitting/overfitting

In [ ]:
sns.lineplot(data = tb_rf_fit, x = 'depth', y = 'f1_train')
sns.lineplot(data = tb_rf_fit, x = 'depth', y = 'f1_test')

# Métodos de Ensemble

Como vimos no exemplo acima, conforme o número de *features* em nosso dataset cresce, árvores de decisão tornam-se extremamente sensíveis à *overfitting*. Para contornar este problema, foram desenvolvidos os **métodos de ensemble**: ao invés de utilizar uma árvore de decisão complexa para realizazr previsões, podemos utilizar muitas árvores simples, reduzindo a chance que qualquer árvore em particular cause overfitting. Veremos hoje as duas principais **estratégias** de *ensemble*: **bagging** e **boosting**.

## Bagging

Os métodos de *bagging* constroem árvores *em paralelo*: ao invés de criar uma árvore complexa, vamos criar muitas árvores de decisão de profundidade limitada (chamadas de *weak learners*). Além de limitar a profundidade máxima de cada *weak learner* vamos treiná-los sobre **amostras do nosso conjunto de treinamento**, ou seja, cada *weak learner* verá uma parte diferente de nosso dataset, garantindo que nenhum deles sofra com **overfitting**.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_fit = RandomForestClassifier(n_estimators = 1500, max_depth= 3)
rf_fit.fit(X_train, y_train)

In [ ]:
tb_fits["pred_prob_rf"] = rf_fit.predict_proba(X_test)[:, 1]
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.scatterplot(
    data=tb_fits, 
    x="lead_time", y="pred_prob_rf", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.scatterplot(
    data=tb_fits, 
    x="adr", y="pred_prob_rf", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');



In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_fits[["lead_time", "adr"]]),
    np.array(tb_fits["is_cancelled"]),
    rf_fit,
    scatter_kwargs={"alpha": 0.001},
)


In [ ]:
tb_simul_lt['pred_prob_rf'] = rf_fit.predict_proba(tb_simul_lt[x_var])[:,1]
tb_simul_adr['pred_prob_rf'] = rf_fit.predict_proba(tb_simul_adr[x_var])[:,1]
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.lineplot(
    data=tb_simul_lt, 
    x="lead_time", y="pred_prob_rf", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.lineplot(
    data=tb_simul_adr, 
    x="adr", y="pred_prob_rf", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');

## Boosting

Enquanto *bagging* utiliza *weak learners* em paralelo, a estratégia de **boosting** os utiliza em série: cada árvore subsequente em nosso modelo é criada para **prever** os erros do *weak learner* anterior!

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat_fit = CatBoostClassifier(
    iterations = 500, depth=4
    )
cat_fit.fit(X_train, y_train, eval_set = (X_test, y_test))

In [ ]:
tb_fits["pred_prob_cat"] = cat_fit.predict_proba(X_test)[:, 1]
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.scatterplot(
    data=tb_fits, 
    x="lead_time", y="pred_prob_cat", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.scatterplot(
    data=tb_fits, 
    x="adr", y="pred_prob_cat", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');

In [ ]:
fig = plt.figure(figsize=(10, 10))
plot_decision_regions(
    np.array(tb_fits[["lead_time", "adr"]]),
    np.array(tb_fits["is_cancelled"]),
    cat_fit,
    scatter_kwargs={"alpha": 0.001},
)


In [ ]:
tb_simul_lt['pred_prob_cat'] = cat_fit.predict_proba(tb_simul_lt[x_var])[:,1]
tb_simul_adr['pred_prob_cat'] = cat_fit.predict_proba(tb_simul_adr[x_var])[:,1]
fig, ax = plt.subplots(1, 2, figsize = (10, 5))
sns.lineplot(
    data=tb_simul_lt, 
    x="lead_time", y="pred_prob_cat", hue = "adr", 
    ax =ax[0], palette='Spectral')
sns.lineplot(
    data=tb_simul_adr, 
    x="adr", y="pred_prob_cat", hue = "lead_time", 
    ax = ax[1], palette = 'Spectral');